In [1]:
import pandas as pd
import numpy as np

## Cleaning the target variable
* Removing observations that can't be used 
    - Loans 
    - Free transfers 
    - Academy promotions
* Converting the fee to an integer
     - Stripping the currency symbol
     - Converting string decimal notation to a multiplier
     - Converting fee to integer on same scale

In [2]:
tf_df = pd.read_csv('data/Top_8_leagues_past_6_windows.csv')

In [3]:
tf_df.drop(labels="Unnamed: 0", axis=1, inplace=True)

In [4]:
tf_df.head()

,player,age,nationality,position,selling_club,previous_league,est_market_value,fee,buying_club,window,year,buying_league
0,Ante Palaversa,18,Croatia,Defensive Midfield,HNK Hajduk Split,Croatia,£495Th.,£5.67m,Manchester City,s_w=w,2018,GB1
1,Ko Itakura,21,Japan,Centre-Back,Kawasaki Frontale,Japan,£630Th.,£990Th.,Manchester City,s_w=w,2018,GB1
2,Yangel Herrera,20,Venezuela,Central Midfield,New York City FC,United States,£900Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1
3,Marlos Moreno,22,Colombia,Right Winger,Clube de Regatas do Flamengo,Brazil,£450Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1
4,Anthony Cáceres,26,Australia,Central Midfield,Melbourne City FC,Australia,£450Th.,"End of loanDec 31, 2018",Manchester City,s_w=w,2018,GB1


In [5]:
# Multiple players have duplicate entries e.g., Nikola Kalinic 2020, Wesley Fofana 2020, Moussa Konaté 2020
# tf_df.loc[tf_df['player']=='Nikola Kalinic',:]

In [6]:
tf_df.shape

(9666, 12)

In [7]:
# Not unusual for players to be transfered multiple times even in same year so only removing duplicates with the
# exact same fee, buying club and selling club in the same year.
tf_df.drop_duplicates(subset=['player','selling_club','buying_club','fee','year'], keep='first', inplace=True)

In [8]:
tf_df.shape

(9378, 12)

In [9]:
tf_df['fee'].value_counts()

free transfer              1139
loan transfer              1088
-                          1010
End of loanJun 30, 2019     988
End of loanJun 30, 2018     932
                           ... 
£6.21m                        1
£59Th.                        1
End of loanJul 16, 2019       1
£16.02m                       1
£23.85m                       1
Name: fee, Length: 627, dtype: int64

### Counting and labeling loan deals

In [10]:
tf_df['loan'] = tf_df['fee'].apply(lambda x: 1 if 'loan' in x.lower() else 0)

In [11]:
tf_df['loan'].value_counts()

1    5056
0    4322
Name: loan, dtype: int64

5056 loan deals.  Not unexpected.

### Counting and labeling free transfers 

In [12]:
tf_df['free'] = tf_df['fee'].apply(lambda x: 1 if 'free' in x.lower() else 0)

In [13]:
tf_df['free'].value_counts()

0    8239
1    1139
Name: free, dtype: int64

In [14]:
#Want to see if there are any other types of free transfers
# pd.options.display.max_rows = 1200
tf_df.loc[tf_df['free'] == 1, 'fee']

20      free transfer
33      free transfer
46      free transfer
86      free transfer
131     free transfer
            ...      
9636    free transfer
9637    free transfer
9638    free transfer
9652    free transfer
9653    free transfer
Name: fee, Length: 1139, dtype: object

Free transfers happen when a contract is run down; contract length is an important determinant of transfer fee but I don't have that information.  Here, while it would be important for clubs to know how much time is left on a contract (as they may be able to use that as leverage to get a better deal) I'll use the model to bench mark the going market rate for a player of that profile.

### In the fee value count: 1010 "-"
### These appear to be internal promotions, e.g., from the U23 team. Investigating and labeling here

In [15]:
# pd.options.display.max_rows = 30
tf_df.loc[tf_df['fee'] == "-", ['player','fee','selling_club','buying_club']]

,player,fee,selling_club,buying_club
16,Callum Hudson-Odoi,-,Chelsea FC U23,Chelsea FC
19,Eddie Nketiah,-,Arsenal FC U23,Arsenal FC
29,Sean Longstaff,-,Newcastle United U23,Newcastle United
42,Kyle Taylor,-,AFC Bournemouth U21,AFC Bournemouth
44,Samir Nasri,-,Disqualification,West Ham United
...,...,...,...,...
9591,Andrey Bokovoy,-,FK Sochi II,FC Sochi
9603,Arsen Adamov,-,Akhmat Grozny II,Akhmat Grozny
9621,Vladimir Kabakhidze,-,FK Tambov II,PFK Tambov
9639,Nikita Repin,-,Rotor 2 Volgograd,Rotor Volgograd


The fee "-" are internal promotions or picking up players without a club for free.  Will not be used in our model

In [16]:
# removing loans, free transfers, academy promotions
tfdf2 = tf_df.loc[(tf_df['fee'] != "-")&(tf_df['loan'] == 0)&(tf_df['free'] == 0), :]

In [17]:
tfdf2.head()

,player,age,nationality,position,selling_club,previous_league,est_market_value,fee,buying_club,window,year,buying_league,loan,free
0,Ante Palaversa,18,Croatia,Defensive Midfield,HNK Hajduk Split,Croatia,£495Th.,£5.67m,Manchester City,s_w=w,2018,GB1,0,0
1,Ko Itakura,21,Japan,Centre-Back,Kawasaki Frontale,Japan,£630Th.,£990Th.,Manchester City,s_w=w,2018,GB1,0,0
10,Christian Pulisic,20,United States,Left Winger,Borussia Dortmund,Germany,£45.00m,£57.60m,Chelsea FC,s_w=w,2018,GB1,0,0
27,Miguel Almirón,24,Paraguay,Attacking Midfield,Atlanta United FC,United States,£13.50m,£21.60m,Newcastle United,s_w=w,2018,GB1,0,0
38,Dominic Solanke,21,England,Centre-Forward,Liverpool FC,England,£9.00m,£19.08m,AFC Bournemouth,s_w=w,2018,GB1,0,0


In [18]:
tfdf2.shape

(2173, 14)

In [19]:
# pd.options.display.max_rows = 20
# tfdf2['previous_league'].value_counts()
# Top 6 countries:
# Italy                   299
# France                  258
# England                 225
# Spain                   222
# Portugal                175
# Germany                 172

Evaluating the value counts of previous league - there are only 1176 observations where a player was bought from a league in the Big 5.  Currently only have detailed statistical profile of players from the top 5 leagues.  Looked to pull in more years and data from more leagues but fbref doesn't have detailed stats for leagues outside the top 5 (except for MLS) or for more than 3 years.  Will have to proceed with what I have understanding this is a bit reduced in scope by virture of data.

In [20]:
#Creating an indicator for bought from a top five league to faciliate merging with statistical data
tfdf2['buying_top_5'] = tfdf2['previous_league'].map(lambda x: 1 if x in ["Italy",'France','England','Spain','Germany'] else 0)

/Users/ericroberts/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
pd.options.mode.chained_assignment = None  # default='warn'

In [22]:
tfdf2['buying_top_5'].value_counts()

1    1176
0     997
Name: buying_top_5, dtype: int64

In [23]:
tfdf3 = tfdf2.loc[(tfdf2['buying_top_5'] == 1), :]

In [24]:
tfdf3.shape

(1176, 15)

### Turning fee column into an integer

In [25]:
tfdf3['fee'].value_counts()

?          69
£1.80m     65
£2.70m     45
£900Th.    42
£4.50m     41
           ..
£324Th.     1
£3.33m      1
£18.81m     1
£90         1
£23.85m     1
Name: fee, Length: 265, dtype: int64

In [26]:
filt =  tfdf3['fee']=='?'
tfdf3.drop(index=tfdf3[filt].index, inplace=True)

In [27]:
tfdf3['fee'].value_counts()

£1.80m     65
£2.70m     45
£900Th.    42
£3.60m     41
£4.50m     41
           ..
£324Th.     1
£3.33m      1
£18.81m     1
£90         1
£23.85m     1
Name: fee, Length: 264, dtype: int64

In [28]:
tfdf3['currency'] = tfdf3['fee'].apply(lambda x: x[0])

In [29]:
tfdf3['currency'].value_counts()

£    1107
Name: currency, dtype: int64

In [30]:
tfdf3['multiplier'] = tfdf3['fee'].str.extract(r'([a-zA-Z]+)')
                                                                         

In [31]:
tfdf3['multiplier'].value_counts()

m     936
Th    168
Name: multiplier, dtype: int64

In [32]:
tfdf3.isna().sum()

player              0
age                 0
nationality         0
position            0
selling_club        0
previous_league     0
est_market_value    0
fee                 0
buying_club         0
window              0
year                0
buying_league       0
loan                0
free                0
buying_top_5        0
currency            0
multiplier          3
dtype: int64

In [33]:
# If there was no multiplier specified assuming it is in the specified units
tfdf3.fillna(value=1, inplace=True)

In [34]:
tfdf3['multiplier'].value_counts()

m     936
Th    168
1       3
Name: multiplier, dtype: int64

In [35]:
tfdf3['fee_numerical'] = tfdf3['fee'].str.extract(r'([0-9]+\.?[0-9]+)')

In [36]:
# convert numeric columns to appropiate dypes
tfdf3 = tfdf3.astype({'fee_numerical':'float'})

In [37]:
tfdf3.loc[tfdf3['multiplier'] == 'm', 'mult_num'] = 1000000
tfdf3.loc[tfdf3['multiplier'] == 'Th', 'mult_num'] = 1000
tfdf3.loc[tfdf3['multiplier'] == 1, 'mult_num'] = 1

In [38]:
tfdf3['fee_final'] = tfdf3['fee_numerical']*tfdf3['mult_num']

In [39]:
tfdf3['fee_final']

10      57600000.0
38      19080000.0
39      12240000.0
57       1530000.0
65      18900000.0
           ...    
9232     2700000.0
9320    18000000.0
9321     5400000.0
9455    10800000.0
9466     4950000.0
Name: fee_final, Length: 1107, dtype: float64

In [40]:
# tfdf3.head()

## Creating variables to index on for merging:
Need to assess and clean variables for this purpose in both dataframes:
* Player name
* Year of stats/year of transfer - using a one year lag (e.g., purchase in 2018 will be modeled with 2017 stats.
    - Note: the winter transfer window is labeled a year early (winter 2018 was the Jan 2019 window). Therefore use the 2017/2018 stats to predict winter/summer 2018 prices
* Columns with same information but different format:
    * club name in stats/selling club in transfers
    * nationality
    * previous_league/league
    * position

In [3]:
statsdf = pd.read_csv('data/Player_stats_top_5_leagues_2017_to_2020.csv')

In [42]:
# tfdf3.head()

In [43]:
# tfdf3['player'].value_counts()

In [44]:
# tfdf3.loc[tfdf3['player']=='Enric Gallego',:]

In [45]:
# statsdf.loc[statsdf['players']=='Enric Gallego',:]

In [4]:
statsdf['xa_net']

0       1.6
1      -0.5
2      -0.4
3      -0.2
4      -0.1
       ... 
8072   -0.7
8073   -0.9
8074    0.0
8075   -0.2
8076    0.7
Name: xa_net, Length: 8077, dtype: float64

In [46]:
statsdf.head()

,players,nationality,team,position,age,birth_year,games,games_start,mins,goals,...,passes_received_pct,miscontrols,dispossessed,passes_left_foot,passes_right_foot,aerials_won,aerials_lost,aerials_won_pct,year,league
0,David Abraham,ar ARG,Eint Frankfurt,DF,31.0,1986.0,27,27,"2,302",0,...,98.6,5.0,6.0,208.0,923.0,62.0,26.0,70.5,2017-2018,Bundesliga
1,Amir Abrashi,al ALB,Freiburg,MF,27.0,1990.0,12,11,850,0,...,86.5,9.0,14.0,18.0,272.0,21.0,19.0,52.5,2017-2018,Bundesliga
2,René Adler,de GER,Mainz 05,GK,32.0,1985.0,14,14,"1,260",0,...,100.0,0.0,0.0,39.0,284.0,0.0,1.0,0.0,2017-2018,Bundesliga
3,Ailton,br BRA,Stuttgart,"DF,FW",22.0,1995.0,5,1,108,0,...,86.8,3.0,0.0,46.0,3.0,2.0,3.0,40.0,2017-2018,Bundesliga
4,Manuel Akanji,ch SUI,Dortmund,DF,22.0,1995.0,11,10,904,0,...,98.1,5.0,3.0,142.0,394.0,17.0,17.0,50.0,2017-2018,Bundesliga


## ASSESSING MISSINGNESS

In [47]:
tfdf3.isna().sum()

player              0
age                 0
nationality         0
position            0
selling_club        0
previous_league     0
est_market_value    0
fee                 0
buying_club         0
window              0
year                0
buying_league       0
loan                0
free                0
buying_top_5        0
currency            0
multiplier          0
fee_numerical       0
mult_num            0
fee_final           0
dtype: int64

In [48]:
pd.options.display.max_rows = 140
statsdf.isna().sum()

players                          0
nationality                      1
team                             0
position                         1
age                              7
birth_year                       7
games                            0
games_start                      0
mins                             0
goals                            0
assists                          0
pens_successful                  0
pens_attempts                    0
yellow_cards                     0
red_cards                        0
goals_per_90                     0
assists_per_90                   0
goals_and_assists_per_90         0
goals_pk_per_90                  0
goals_assists_pk_per_90          0
xg                              10
npxp                            10
xa                              10
xg_per90                        20
xa_per90                        20
xg_xa_per90_list                20
npxg_per90_list                 20
npxg_xa_per90                   20
full_90s_played     

* variables to use fillna:
    - progressive_passes

In [49]:
#Following variables are missing hundreds of observations, are duplicates between scraped pages or will need to 
#recalculated after combining performance of one player for multiple teams.
# Dropping columns
statsdf.drop(labels=['goals_per_90', 'assists_per_90', 'goals_and_assists_per_90', 'goals_pk_per_90', 
                     'goals_assists_pk_per_90', 'xg_per90', 'xa_per90', 'xg_xa_per90_list', 'npxg_per90_list', 
                     'npxg_xa_per90', 'shots_total_per90', 'shots_on_target_per90',
                     'aerials_won_pct', 'shots_on_target_pct','goals_per_shot','goals_per_shot_on_target',
                    'avg_shot_dist','npxg_per_shot','pass_percent','pass_percent_short','pass_percent_medium',
                    'pass_percent_long','dribble_tackles_pct','pressure_regain_pct','dribbles_completed_pct', 
                    'passes_received_pct','goals.1','pens_successful.1','pens_attempts.1','xg.1','npxg_per_shot',
                    'xg_net','npxg_net','full_90s_played.1','assists.1','xa.1','full_90s_played.2','sca_per90',
                     'gca_per90'], axis=1, inplace=True)

In [50]:
#Assessing remaining missing variables 
pd.options.display.max_rows = 140
statsdf.isna().sum()

players                        0
nationality                    1
team                           0
position                       1
age                            7
birth_year                     7
games                          0
games_start                    0
mins                           0
goals                          0
assists                        0
pens_successful                0
pens_attempts                  0
yellow_cards                   0
red_cards                      0
xg                            10
npxp                          10
xa                            10
full_90s_played                0
shots_total                    7
shots_on_target                0
npxg                          10
passes_completed              10
passes_attempted              10
passes_total_dist             10
passes_prog_dist              10
passes_completed_short        10
passes_attempted_short        10
passes_completed_medium       10
passes_attempted_medium       10
passes_com

In [51]:
statsdf2 = statsdf.fillna(value='?')

In [52]:
# Lots of variables are missing 10 or 20 observations.  Guessing these are all on the same player,
# likely with few appearances and therefore no statistics to report.  Confirming here.
pd.options.display.max_columns = 116
statsdf2.loc[statsdf2['passes_completed'] == '?',:]

,players,nationality,team,position,age,birth_year,games,games_start,mins,goals,assists,pens_successful,pens_attempts,yellow_cards,red_cards,xg,npxp,xa,full_90s_played,shots_total,shots_on_target,npxg,passes_completed,passes_attempted,passes_total_dist,passes_prog_dist,passes_completed_short,passes_attempted_short,passes_completed_medium,passes_attempted_medium,passes_completed_long,passes_attempted_long,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes,sca,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_defense,gca,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_defense,gca_og_for,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,dribble_tackles,dribble_vs,dribbled_past,pressures,pressure_regains,pressures_def_3rd,pressures_mid_3rd,pressures_att_3rd,blocks,blocked_shots,blocked_shots_saves,blocked_passes,interceptions,tackles_interceptions,clearances,errors,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,pass_targets,passes_received,miscontrols,dispossessed,passes_left_foot,passes_right_foot,aerials_won,aerials_lost,year,league
1821,Aiden O'Neill,au AUS,Burnley,MF,19,1998,1,0,1,0,0,0,0,0,0,?,?,?,0.0,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,Premier-League
3836,Christian Kouakou,ci CIV,Caen,FW,27,1991,1,0,13,0,0,0,0,0,0,?,?,?,0.1,0,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2018-2019,Ligue-1
4505,Ahmad Ngouyamsa,cm CMR,Dijon,DF,18,2000,1,0,55,0,0,0,0,1,0,?,?,?,0.6,0,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,1,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2019-2020,Ligue-1
4774,Christian Rutjens,?,Benevento,?,?,?,1,0,1,0,0,0,0,0,0,?,?,?,0.0,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,Serie-A
4832,Emanuele Torrasi,it ITA,Milan,MF,18,1999,1,0,6,0,0,0,0,0,0,?,?,?,0.1,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,Serie-A
5785,Álex Centelles,es ESP,Udinese,DF,20,1997,8,6,583,0,0,0,0,4,0,?,?,?,6.5,4,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,7,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,13,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2018-2019,Serie-A
6431,Sergio Arribas,es ESP,Leganés,FW,22,1995,1,0,24,0,0,0,0,0,0,?,?,?,0.3,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,La-Liga
6760,Álex Mula,es ESP,Málaga,"MF,FW",21,1996,11,7,560,0,0,0,0,1,0,?,?,?,6.2,?,6,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,La-Liga
6767,Fahad Al-Muwallad,sa KSA,Levante,"FW,MF",20,1996,2,0,29,0,0,0,0,0,0,?,?,?,0.3,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,La-Liga
6948,Asier Villalibre,es ESP,Athletic Club,FW,19,1997,1,1,90,0,0,0,0,0,0,?,?,?,1.0,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,0,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,2017-2018,La-Liga


In [53]:
#Confirmed - of the players with missing values 7 have 1 appearance, one 2, one 8 and one 11 (the latter two 
#greater than 6 full 90's) going to zero fill
statsdf.fillna(value=0, inplace=True)

In [54]:
#Not sure why but Christian Rutjens (index=4774 has no information)
statsdf.drop(index=4774, inplace=True)

In [55]:
statsdf.isna().sum()

players                       0
nationality                   0
team                          0
position                      0
age                           0
birth_year                    0
games                         0
games_start                   0
mins                          0
goals                         0
assists                       0
pens_successful               0
pens_attempts                 0
yellow_cards                  0
red_cards                     0
xg                            0
npxp                          0
xa                            0
full_90s_played               0
shots_total                   0
shots_on_target               0
npxg                          0
passes_completed              0
passes_attempted              0
passes_total_dist             0
passes_prog_dist              0
passes_completed_short        0
passes_attempted_short        0
passes_completed_medium       0
passes_attempted_medium       0
passes_completed_long         0
passes_a

### Converting mins to a numeric variable

In [56]:
statsdf['mins_num'] = statsdf['mins'].map(lambda x: x.replace(",",""))

In [57]:
#convert to int
statsdf = statsdf.astype({'mins_num':'int'})

In [58]:
statsdf['mins_num'].describe()

count    8076.000000
mean     1315.177935
std       986.559807
min         1.000000
25%       393.000000
50%      1197.500000
75%      2117.000000
max      3420.000000
Name: mins_num, dtype: float64

### Years
* Year of stats/year of transfer - using a one year lag (e.g., purchase in 2018 will be modeled with 2017 stats.
    - Note: the winter transfer window is labeled a year early (winter 2018 was the Jan 2019 window). Therefore use the 2017/2018 stats to predict winter/summer 2018 prices

In [59]:
print(tfdf3['year'].value_counts())
print(tfdf3['window'].value_counts())
print(statsdf['year'].value_counts())

2019    435
2018    401
2020    271
Name: year, dtype: int64
s_w=s    976
s_w=w    131
Name: window, dtype: int64
2019-2020    2732
2017-2018    2686
2018-2019    2658
Name: year, dtype: int64


In [60]:
# tfdf3.head(20)

In [61]:
tfdf3['index_year'] = tfdf3['year']

In [62]:
#renaming year as transfer year so that after merging the column is perserved 
#and interpretable
tfdf3.rename(columns={'year':'transfer_year'}, inplace=True)

In [63]:
#Select the first 4 digits (the first year) for the stats years
statsdf['index_year'] = statsdf['year'].apply(lambda x: x[0:4])

In [64]:
#convert to int
statsdf = statsdf.astype({'index_year':'int'})

In [65]:
#Adding 1 to faciliate matching of the lag
#index_year will now match up the transfer to the performance stats lagged by one year
statsdf['index_year'] = statsdf['index_year']+1

In [66]:
#renaming year as stats_year so that after merging the column is perserved 
#and interpretable
statsdf.rename(columns={'year':'stats_year'}, inplace=True)

In [67]:
statsdf['index_year'].value_counts()

2020    2732
2018    2686
2019    2658
Name: index_year, dtype: int64

### Player name

In [68]:
tfdf3['player'].nunique()

1024

In [69]:
statsdf['players'].value_counts()

Raúl García          8
Adama Traoré         6
Naldo                6
Valentin Eysseric    6
Marcelo              6
                    ..
Moritz Bauer         1
Simon Mignolet       1
Fabian Bredlow       1
Joe Bennett          1
Ariday Cabrera       1
Name: players, Length: 4018, dtype: int64

In [70]:
statsdf.loc[statsdf['players']=='Raúl García',['players','nationality','team','position','age','birth_year','index_year','stats_year']]

,players,nationality,team,position,age,birth_year,index_year,stats_year
6578,Raúl García,es ESP,Leganés,DF,28.0,1989.0,2018,2017-2018
6579,Raúl García,es ESP,Athletic Club,"MF,FW",31.0,1986.0,2018,2017-2018
7362,Raúl García,es ESP,Athletic Club,"MF,FW",32.0,1986.0,2019,2018-2019
7363,Raúl García,es ESP,Girona,DF,29.0,1989.0,2019,2018-2019
7364,Raúl García,es ESP,Leganés,DF,29.0,1989.0,2019,2018-2019
7927,Raúl García,es ESP,Athletic Club,"MF,FW",33.0,1986.0,2020,2019-2020
7928,Raúl García,es ESP,Getafe,DF,30.0,1989.0,2020,2019-2020
7929,Raúl García,es ESP,Valladolid,"DF,MF",30.0,1989.0,2020,2019-2020


* Looking up Raul Garcia - there is the raul garcia that has been at Bilbao for years.
* The other 5 entries here are the same Raul Garcia.  He spent time on loan and therefore time at two clubs in 2019 and 2020.  
* Options:
    - combine the stats from the same year (all % columns will be unuseable)
    - Only use the stats for the selling club.  This is of course limited, but no more limited than not considering more than one season of work.

In [71]:
tfdf3['index_name'] = tfdf3['player']

In [72]:
statsdf['index_name'] = statsdf['players']

# Merging transfer and stats dataframes by index

## Original idea
* Merging on name, nationality, selling club and year (stats year lagged one behind sell year)
* Did not use age because the timing of the entry in either database was unclear so couldn't get a consistent number lagged or otherwise
## Problems with this approach
* There are players that are transferred multiple times in the same year
    - e.g.,  Omar Mascarell. Madrid exercised a buy back and then sold (one stats year, two transfers)
    - e.g., Marc Cucurella. Loaned to Eibar 2018/2019, option to make permanent in summer 2019.  Barca rebought 16 days later and loaned him to Getafe.  Permanently sold to Getafe summer 2020.
    * IMPACT: We might assume buybacks are set based on the performance prior to their loan but there is nothing in the dataset indicating options to buy (or release fees).  Treating these as independent transactions based on the same performance year.
* There are players in the stats with multiple observations per year:
    - Loans/January moves/extended window, e.g., Naldo, Adama Traoré, Rafinha - One full year of stats spread across multiple teams
    - Mulitple people with same name, nationality, e.g., Naldo which means we can't just sum by player and year
    - IMPACT: we can't match based on club because of the multi club problem and we can't sum by player and year because of the multi player problem.  
* Solution: 
    - Sum stats df by name, year and dob
    - This will eliminate all string var from df (will lose nationality, team, position and league) - therefore only left with name and year to merge with transfer data
    - Individual player profiles on tranfermarkt, have identifying numbers so can't automate the scrap e.g., https://www.transfermarkt.co.uk/nikola-kalinic/profil/spieler/36371
    - Checked calculating birth year by subtracting age from transfer year - not always correct and likely to lose more data this way than by eliminating ambiguously named/transfer year players. 
    - 2018 6 duplicated names

In [73]:
statsdf.set_index(['index_name','birth_year','index_year'], inplace=True)

In [74]:
statsdf3 = statsdf.sum(level=['index_name','birth_year','index_year'])

In [75]:
statsdf3.head()

,,,age,games,games_start,goals,assists,pens_successful,pens_attempts,yellow_cards,red_cards,xg,npxp,xa,full_90s_played,shots_total,shots_on_target,npxg,passes_completed,passes_attempted,passes_total_dist,passes_prog_dist,passes_completed_short,passes_attempted_short,passes_completed_medium,passes_attempted_medium,passes_completed_long,passes_attempted_long,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes,sca,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,sca_defense,gca,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_defense,gca_og_for,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,dribble_tackles,dribble_vs,dribbled_past,pressures,pressure_regains,pressures_def_3rd,pressures_mid_3rd,pressures_att_3rd,blocks,blocked_shots,blocked_shots_saves,blocked_passes,interceptions,tackles_interceptions,clearances,errors,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,players_dribbled_past,nutmegs,carries,carry_distance,carry_progressive_distance,pass_targets,passes_received,miscontrols,dispossessed,passes_left_foot,passes_right_foot,aerials_won,aerials_lost,mins_num
index_name,birth_year,index_year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
David Abraham,1986.0,2018,31.0,27,27,0,2,0,0,3,0,0.5,0.5,0.4,25.6,12.0,1,0.5,1081.0,1313.0,25128.0,7959.0,241.0,275.0,588.0,655.0,243.0,369.0,1.6,6.0,66.0,3.0,2.0,91.0,16.0,12.0,1.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,43.0,28.0,31.0,11.0,1.0,24.0,41.0,17.0,221.0,74.0,148.0,66.0,7.0,39.0,16.0,0.0,23.0,23.0,66,130.0,1.0,1561.0,151.0,830.0,844.0,43.0,13.0,1526.0,13.0,14.0,13.0,0.0,955.0,6200.0,3493.0,903.0,890.0,5.0,6.0,208.0,923.0,62.0,26.0,2302
Amir Abrashi,1990.0,2018,27.0,12,11,0,0,0,0,2,0,0.2,0.2,0.5,9.4,5.0,0,0.2,260.0,335.0,4482.0,1183.0,124.0,157.0,104.0,124.0,26.0,42.0,-0.5,4.0,20.0,4.0,1.0,30.0,12.0,10.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,26.0,14.0,22.0,1.0,11.0,20.0,9.0,215.0,57.0,71.0,118.0,26.0,20.0,5.0,0.0,15.0,11.0,48,19.0,2.0,471.0,28.0,122.0,288.0,80.0,9.0,467.0,5.0,11.0,7.0,0.0,277.0,1277.0,553.0,266.0,230.0,9.0,14.0,18.0,272.0,21.0,19.0,850
René Adler,1985.0,2018,32.0,14,14,0,0,0,0,0,0,0.0,0.0,0.4,14.0,0.0,0,0.0,241.0,386.0,8770.0,5681.0,20.0,22.0,97.0,102.0,124.0,262.0,-0.4,1.0,8.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4.0,0.0,442.0,334.0,440.0,3.0,0.0,0.0,333.0,0.0,0.0,0.0,0.0,237.0,1236.0,652.0,171.0,171.0,0.0,0.0,39.0,284.0,0.0,1.0,1260
Ailton,1995.0,2018,22.0,5,1,0,0,0,0,0,0,0.0,0.0,0.2,1.2,1.0,0,0.0,45.0,62.0,884.0,297.0,20.0,26.0,16.0,18.0,8.0,15.0,-0.2,2.0,3.0,2.0,2.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,3.0,2.0,0.0,3.0,7.0,4.0,41.0,12.0,21.0,15.0,5.0,5.0,0.0,0.0,5.0,1.0,6,1.0,0.0,83.0,3.0,23.0,50.0,14.0,3.0,72.0,1.0,3.0,1.0,0.0,48.0,297.0,189.0,53.0,46.0,3.0,0.0,46.0,3.0,2.0,3.0,108
Manuel Akanji,1995.0,2018,22.0,11,10,0,0,0,0,2,0,0.2,0.2,0.1,10.0,5.0,2,0.2,521.0,604.0,11385.0,3396.0,141.0,161.0,290.0,314.0,88.0,122.0,-0.1,1.0,39.0,3.0,0.0,39.0,6.0,4.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,12.0,14.0,6.0,0.0,3.0,8.0,5.0,122.0,46.0,64.0,52.0,6.0,14.0,4.0,0.0,10.0,7.0,27,58.0,0.0,720.0,56.0,312.0,422.0,65.0,10.0,688.0,6.0,7.0,6.0,0.0,477.0,3592.0,2112.0,486.0,477.0,5.0,3.0,142.0,394.0,17.0,17.0,904


In [76]:
print(statsdf.shape)
print(statsdf3.shape)

(8076, 98)
(7732, 91)


In [77]:
statsdf3.reset_index(inplace=True)

In [ ]:
# Checked calculation against some random profiles - inaccurate.
# tfdf3['birth_year'] = tfdf3['transfer_year'] - tfdf3['age']
# tfdf3.loc[:,['player','age','birth_year']]

# Preping for merge
* Because I can't easily scrape dob or calculate birth year in the transfer df in the below I check how many names are repeated in the statsdf by year (the merging levels) and cross-referencing against entries in the transfer df 

* 2018: Naldo sold from schalke to monaco (delete index 6505)
* Not sold in year: Raúl García, Juanfran, Rafinha, Marcelo, Rafael sold

* 2019: 
* Not sold in year: Nacho, Allan, Manu García, Adama Traoré, Marcelo, Raúl García, Sergio Álvarez, Juanfran

* 2020:
* Not sold in year: Javi Martinez, Adama Traoré, Sergio Álvarez, Marcelo, Jonas Hofmann, Pedro, Javi López, Raúl García, Nacho, João Pedro, Rafael, Ibrahim Cissé

In [123]:
statsdf3.drop(index=6505, inplace=True)

In [243]:
#assessing individual names to make them unambiguous
# tfdf3['index_name'].value_counts()

In [244]:
# tfdf3.loc[tfdf3['index_name']=='Coke']

In [245]:
# statsdf3.loc[statsdf3['index_name']=='Coke']

In [133]:
#Gabriel uses different name between df's
tfdf3.loc[(tfdf3['index_name']=='Gabriel')&(tfdf3['selling_club']=='LOSC Lille'), ['player','index_name']] = 'Gabriel Dos Santos'

In [137]:
#Danilo Larangeira
tfdf3.loc[(tfdf3['index_name']=='Danilo')&(tfdf3['selling_club']=='Udinese Calcio'), ['player','index_name']] = 'Danilo Larangeira'

In [202]:
#Arthur Melo
tfdf3.loc[(tfdf3['index_name']=='Arthur')&(tfdf3['selling_club']=='FC Barcelona'), ['player','index_name']] = 'Arthur Melo'

In [208]:
#Thiago Alcántara
tfdf3.loc[(tfdf3['index_name']=='Thiago')&(tfdf3['selling_club']=='Bayern Munich'), ['player','index_name']] = 'Thiago Alcántara'

In [216]:
# Raphael Dias Belloli
tfdf3.loc[(tfdf3['index_name']=='Raphinha')&(tfdf3['selling_club']=='Stade Rennais FC'), ['player','index_name']] = 'Raphael Dias Belloli'

In [222]:
# Kepa Arrizabalaga
tfdf3.loc[(tfdf3['index_name']=='Kepa')&(tfdf3['selling_club']=='Athletic Bilbao'), ['player','index_name']] = 'Kepa Arrizabalaga'

In [228]:
#Chicarito/ Javier Hernández
tfdf3.loc[(tfdf3['index_name']=='Chicharito')&(tfdf3['selling_club']=='West Ham United'), ['player','index_name']] = 'Javier Hernández'

In [237]:
#Fernando Marçal
tfdf3.loc[(tfdf3['index_name']=='Marçal')&(tfdf3['selling_club']=='Olympique Lyon'), ['player','index_name']] = 'Fernando Marçal'

In [159]:
#Vagner Gonçalves - he spent 2018/2019 2019/2020 in second division (no stats)
tfdf3.loc[(tfdf3['index_name']=='Vagner')&(tfdf3['selling_club']=='AS Saint-Étienne'), ['player','index_name']] = 'Vagner Gonçalves'

In [242]:
#Fixing data entry errors for age:
statsdf3.loc[(statsdf3['index_name']=='Suso')&(statsdf3['index_year']==2020), 'age'] = 25
statsdf3.loc[(statsdf3['index_name']=='Sandro')&(statsdf3['index_year']==2019), 'age'] = 29
statsdf3.loc[(statsdf3['index_name']=='Javier Hernández')&(statsdf3['index_year']==2020), 'age'] = 31
statsdf3.loc[(statsdf3['index_name']=='Coke')&(statsdf3['index_year']==2018), 'age'] = 30

In [246]:
df = tfdf3.merge(statsdf3, on=['index_name','index_year'])

In [247]:
df.shape

(678, 114)

In [248]:
df.to_csv('data/transfers_and_stats_merged.csv')

## Variable creation no longer needed
* Harmonizing nationality and selling club

Creating a dictionary of country abbreviations to create a new column in the stats df that is the full name of the nation to facilitate index matching with nationality in the transfer df

In [ ]:
#from: https://www.realifewebdesigns.com/web-marketing/abbreviations-countries.asp
# country_abb = pd.read_excel('data/country_abbreviations.xlsx', header=None)

In [ ]:
#format: "AF = Afghanistan"
#extracting the country code abbreviation and country name as key and value for dictionary
# country_abb['key'] = country_abb[0].apply(lambda x: x[0:2].lower())
# country_abb['value'] = country_abb[0].apply(lambda x: x[4:])

In [ ]:
#dropping original column
# country_abb.drop(labels=0, axis=1, inplace=True)

In [ ]:
#Convert series to lists to facilitate creation of dictionary
# key_list = list(country_abb['key'])
# value_list = list(country_abb['value'])

In [ ]:
# country_dict = {}
# for n in range(len(key_list)):
#     country_dict[key_list[n]] = value_list[n]

In [ ]:
#Extracting the abbreviation from the nationality column (formatting as "ar ARG")
# statsdf['nationality_abb'] = statsdf['nationality'].str.extract(r'([a-z]+)')

In [ ]:
#the entries for guadelupe were missing the two letter code
#French guiana's code is GUF, the one GYF is a data entry error correcting here
#explains the nunique discrepancy 
# statsdf.loc[statsdf['nationality'] == ' GPE', 'nationality_abb'] = 'gp' 
# statsdf.loc[statsdf['nationality'] == ' GYF', 'nationality_abb'] = 'gf'

In [ ]:
# print(statsdf['nationality'].nunique())
# print(statsdf['nationality_abb'].nunique())

# print(statsdf['nationality'].value_counts())
# print(statsdf['nationality_abb'].value_counts())

In [ ]:
#Manually entering missing abbreviations from the list from the website to faciliate recoding:
# country_dict['cw'] = 'Curacao'
# country_dict['eng'] = 'England'
# country_dict['is'] = 'Iceland'
# country_dict['rs'] = 'Serbia'
# country_dict['xk'] = 'Kosovo'
# country_dict['wal'] = 'Wales'
# country_dict['sco'] = 'Scotland'
# country_dict['nir'] = 'Northern Ireland'
# country_dict['me'] = 'Montenegro'

In [ ]:
# statsdf['index_nationality'] = statsdf['nationality_abb'].map(lambda x: country_dict[x])

In [ ]:
# tfdf3['index_nationality'] = tfdf3['nationality']

## Indexing on selling club in transfer data and club in stats data

* The previous league column in tfdf3 is more correctly previous country.  Therefore includes purchases from lower leagues and academies not just top tier.  Therefore tfdf3 has more clubs than appear in the stats dataset.  The team in statsdf is the more generic version.  
* Import a dictionary with generic name as key and full name as value from pyfiles to add to the statsdf for index matching.

In [ ]:
# import py_files.team_dictionary as team_dict

In [ ]:
# team_dict = team_dict.team_dict

In [ ]:
# statsdf['index_selling_club'] = statsdf['team'].map(lambda x: team_dict[x])

In [ ]:
# tfdf3['index_selling_club'] = tfdf3['selling_club']